# TreeStructure DS creation

![BinaryTree](tree.png)

#### Example

0. : _does it move?_
1. : _does it swim?_
2. : _does it have bark?_
3. : _is it a mammal?_
4. : _is it vertebrate?_
5. : _it is a conifer?_
6. : _does it have sepals?_
7. : Whale
8. : Salmon
9. : Wolf
10. : Butterfly
11. : Larch
12. : Oak tree
13. : Rose
14. : Weed

**Def**. _Pattern_: collection of _all_ the node values of the array-represented tree. 

As an example, to the non-leaves nodes are associated decision rules, intended to discriminate samples (e.g.: _does the object move?_, which can be answered with _yes_ or _no_, $\pm 1$, is the primal decision rule, i.e. axis along which one can set distinctions).

The leaves nodes represent elements of a given category. It is a simple and far-from-reality example.

In the following code the following criteria are implemented:

0. The probabilistic threshold is fixed a priori. The smaller its value, the less variability in the data set.
1. The root node has as value a random variable attaining the values $\pm 1$ with probability $p = 0.5$.
2. Root's children attain values $+1$ or $-1$ in a mutually exclusive fashion. The following convention is adopted: _if the root node attains the value $+1$, then the left child inherits the same value. Else, the left child attains the value $-1$ and the right child has assigned the value $+1$_. 
3. From the third level (children of root's children), the progeny of any node that has value $-1$ also has to have $-1$ value. On the other hand, if one node has value $+1$, its value is inherited (again mutually exclusively) by its children according to a probabilistic decision rule. This enforces the uniqueness of the final pattern, consistently with the ancestry of the leaves. 

The aforementioned probabilistic decision rule is a Metropolis-like criterion: Sample a random variable $p \sim U([0,1])$, then, given the probabilistic threshold $\epsilon$,

* If $p > \epsilon$, the left child inherits the $+1$ value, and the right child, alongside with its progeny, assume the opposite value;
* Else, is the right child to assume the value $+1$.

Subsequently, it is possible to choose the value of the `lev` variable. This one sets the level, at which stop the **distinction granularity**. Such a data set has a hierarchical structure. As such, it is possible to set coarse grained distinctions (e.g. _the object moves/does not move_) and fine grained distinctions (_does the object belong to mammals or not?_). The more detailed the structure of the tree itself, the more realistic is the pattern differentiation.

As an example consider the above depicted tree. It has depth $D = 4$, that is $D = \max{\{L\}} + 1$ being $L = 0, \dots, D-1$ index the levels. This latter is indeed the `lev` variable. Set `lev = 2` (in mathematical terms $L = 2$), the code then builds the label matrix comparing **all the first** $B_f^{L+1}-2 = 2^3 - 2 = 0, \dots, 6$ nodes. The rationale is this: Each node bears the outcome of a decision (a semantic distinction), then at level $L = 2$ one can at most differentiate in all the classes that can be identified by these distinctions.

For the sake of simplicity, consider $L = 1$. At this level, one can count the nodes up to 2, this latter comprehended. Node 1 sets the distinction _does it swim?_ and node 2 says whether an object _has the bark or not_. Thus, at this level one can distinguish at most 4 classes. 

The higher the level chosen, the finer grainded the granulometry.


##### Why is the data set subsequently messed up by random values flip?

The above desscribed data generation process resolves to the creation of a **linearly separable** dataset, owing to the **sharp** distinctions assessable. If one arbitrarily and randomly flips some of the data matrix values, then he (or she) may end up with a pine tree that can swim, obvioulsy a surreal scenario. This translates, during the stage of learning, in a non-triavial assimilation of information by the learning artificial agent, rendering lesser accuracy and higher training speed.

In [10]:
%%time
import numpy as np
from numpy import random

import pickle
from math import floor, ceil


class BinaryTreeDataSet:
    
    def __init__(self,Bf,D,M):
                              # Bf : branching factor
                              # D : livelli dell'albero
        self.N = Bf**(D) - 1  # tutti i nodi
        self.n = Bf**(D-1) -1 # tutti i nodi NON foglie
        self.P = Bf**(D-1)    # tutti i nodi foglie
        self.M = M            # quanti pattern (data items)
        
        print('\nTree characts:\n')
        print('   Nodes (feats) : N = ',self.N)
        print('Nodes NOT leaves : n = ',self.n)
        print('    Nodes leaves : P = ',self.P)
        print('        Patterns : M = ',self.M)
    #end

    def patternGenerator(self):
        
        N = self.N
        n = self.n
    
        tree = np.zeros(N)
        outcomes = [-1,1]
        e = 0.3
        tree[0] = outcomes[random.randint(0,2)]
        
        
        #p = random.rand()
        #if (p >= 0.5):
        #    tree[1] = tree[0]
        #    tree[2] = (-1.) * tree[0]
        #else: 
        #    tree[1] = (-1.) * tree[0]
        #    tree[2] = tree[0]
        #endif
        
        if (tree[0] == 1):
            tree[1] = 1
            tree[2] = -1
        else:
            tree[1] = -1
            tree[2] = 1
        #end
        
        for k in range(1,n):
            if (tree[k] == 1):
                p = random.rand()
                if (p > e):
                    tree[2*k + 1] = tree[k]
                    tree[2*k + 2] = (-1.) * tree[k]
                else:
                    tree[2*k + 1] = (-1.) * tree[k]
                    tree[2*k + 2] = tree[k]
                #endif
            else:
                tree[2*k + 1] = -1
                tree[2*k + 2] = -1
            #endif
        #enddo
        
        return tree
    #end


    def dataSetGenerator(self):
        
        #print('in dataset generator')
        N = self.N; M = self.M
        Y = np.zeros((M,N))
        
        for m in range(M):
            Y[m,:] = self.patternGenerator()
        #end
        
        return Y
    #end
    
#end


# ------------------ Deployment

Bf = 2
D = 5
M = 2000

treeData = BinaryTreeDataSet(Bf,D,M)
X = treeData.dataSetGenerator()
print("\n",type(X),"\n",X)

# ATTENZIONE! lev < stretto di D!
lev = 2

Y = np.eye(X.shape[0])
print("DataSet: {} data entry having {} features each\n".format(X.shape[0], X.shape[1]))

def view1D(a): # a is array
    a = np.ascontiguousarray(a)
    void_dt = np.dtype((np.void, a.dtype.itemsize * a.shape[1]))
    return a.view(void_dt).ravel()
#enddef

# Get 1D view
lowBound = Bf**lev - 1
uppBound = Bf**(lev + 1) - 2

X_ = X[:,  : uppBound+1 ]
a1D = view1D(X_)

# Perform broadcasting to get outer equality match
mask = a1D[:,None]==a1D

# Get indices of pairwise matches
n = len(mask)
mask[np.tri(n, dtype=bool)] = 0
idx = np.argwhere(mask)

# Run loop to assign equal rows in Y
for (i,j) in zip(idx[:,0],idx[:,1]):
    Y[j] = Y[i]
#enddo

check = np.zeros(Y.shape[0])
listZeroCol = []
listNoZeroCol = []
for i in range(Y.shape[1]):
    if (np.all( (Y[:,i] == check), axis = 0)):
        #print(i)
        listZeroCol.append(i)
    #endif
#enddo

listNoZeroCol = [i for i in range(Y.shape[1]) if i not in listZeroCol]
Y = Y[:,listNoZeroCol]

print(Y.shape," quindi il numero di classi diverse è {}".format(Y.shape[1]))
print(" ")
print(X)
print(" ")
print(Y)
print(" ")




Tree characts:

   Nodes (feats) : N =  31
Nodes NOT leaves : n =  15
    Nodes leaves : P =  16
        Patterns : M =  2000

 <class 'numpy.ndarray'> 
 [[ 1.  1. -1. ... -1. -1. -1.]
 [-1. -1.  1. ... -1. -1. -1.]
 [ 1.  1. -1. ... -1. -1. -1.]
 ...
 [-1. -1.  1. ... -1. -1. -1.]
 [ 1.  1. -1. ... -1. -1. -1.]
 [-1. -1.  1. ... -1. -1. -1.]]
DataSet: 2000 data entry having 31 features each

(2000, 4)  quindi il numero di classi diverse è 4
 
[[ 1.  1. -1. ... -1. -1. -1.]
 [-1. -1.  1. ... -1. -1. -1.]
 [ 1.  1. -1. ... -1. -1. -1.]
 ...
 [-1. -1.  1. ... -1. -1. -1.]
 [ 1.  1. -1. ... -1. -1. -1.]
 [-1. -1.  1. ... -1. -1. -1.]]
 
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]]
 
Wall time: 9.14 s


In [32]:
!echo %cd%

DataSet = [X, Y]

import pickle

fileID = open(r'C:\Users\Matteo\Desktop\MasterThesis\newThread\DS_model\DataSet_list_clean.pkl', 'wb')
pickle.dump(DataSet, fileID)
fileID.close()

!dir

C:\Users\Matteo\Desktop\MasterThesis\newThread\DS_model
 Il volume nell'unitÃ  C Ã¨ TI31237300A
 Numero di serie del volume: 5685-0C6A

 Directory di C:\Users\Matteo\Desktop\MasterThesis\newThread\DS_model

01/06/2019  16:49    <DIR>          .
01/06/2019  16:49    <DIR>          ..
31/05/2019  16:17    <DIR>          .ipynb_checkpoints
01/06/2019  16:49            16,047 BinaryTree__DS.ipynb
31/05/2019  08:20            20,975 Complete__pipeline.ipynb
01/06/2019  16:50           560,208 DataSet_list_clean.pkl
30/05/2019  10:39           152,208 DataSet_list_noised.pkl
31/05/2019  11:41            28,719 L1.png
31/05/2019  11:41            33,259 L2.png
01/06/2019  13:27         1,193,958 NetworkDraw.ipynb
01/06/2019  11:01                 0 network_exmp_OUT.txt
01/06/2019  13:26           800,307 NN_model.ipynb
01/06/2019  16:47            12,735 PCA_16cl.png
01/06/2019  16:46            12,415 PCA_2cl.png
01/06/2019  16:47            12,317 PCA_4cl.png
01/06/2019  16:48            13

# Data set alteration (to render non linearly separability)

In [11]:
!echo %cd%

MaxFlip = floor(X.shape[0]*X.shape[1]/5.)
X_ = X

if (MaxFlip < 5):
    print("non flippi niente eh")
#end

for k in range(MaxFlip):
    rowIdx = np.random.randint(0,X.shape[0])
    colIdx = np.random.randint(0,X.shape[1])
    #print(rowIdx,colIdx)
    X_[rowIdx,colIdx] = (-1.) * X[rowIdx,colIdx]
#end

DataSet = [X_, Y]

fileID = open(r'C:\Users\Matteo\Desktop\MasterThesis\newThread\DS_model\DataSet_list_noised.pkl', 'wb')
pickle.dump(DataSet, fileID)
fileID.close()

!dir

C:\Users\Matteo\Desktop\MasterThesis\newThread\DS_model
 Il volume nell'unitÃ  C Ã¨ TI31237300A
 Numero di serie del volume: 5685-0C6A

 Directory di C:\Users\Matteo\Desktop\MasterThesis\newThread\DS_model

02/06/2019  09:38    <DIR>          .
02/06/2019  09:38    <DIR>          ..
31/05/2019  16:17    <DIR>          .ipynb_checkpoints
02/06/2019  09:38            16,929 BinaryTree__DS.ipynb
31/05/2019  08:20            20,975 Complete__pipeline.ipynb
01/06/2019  16:50           560,208 DataSet_list_clean.pkl
02/06/2019  09:39           560,208 DataSet_list_noised.pkl
31/05/2019  11:41            28,719 L1.png
31/05/2019  11:41            33,259 L2.png
01/06/2019  19:38         1,588,006 NetworkDraw.ipynb
01/06/2019  11:01                 0 network_exmp_OUT.txt
01/06/2019  19:39           707,315 NN_model.ipynb
01/06/2019  17:46         1,098,087 NN_untrained.png
02/06/2019  09:36           111,173 PCA_16cl_noise.png
02/06/2019  09:34            72,295 PCA_2cl_noise.png
02/06/2019  09